# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect("DSN=College;"
                      "Server=DESKTOP-E6QQ4LT\SQLEXPRESS;"
                      "Database=College;"
                      "Trusted_Connection=yes;")


In [3]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
students.head(0)

,Student_id,First_name,Last_name,Gender


In [4]:
departments = pd.read_sql("SELECT * FROM Departments;",conn)
departments.head(0)

,Department_id,Department_name


In [5]:
courses = pd.read_sql("SELECT * FROM Courses;",conn)
courses.head(0)

,Course_id,Course_name,Department_id,Teacher_id


In [6]:
classroom = pd.read_sql("SELECT * FROM Classroom;",conn)
classroom.head(0)

,Course_id,Student_id,Degree


In [7]:
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)
teachers.head(0)

,Teacher_id,First_name,Last_name,Gender


# 1. Count the number of students on each department

In [8]:
#inner join classroom with courses
classroom_courses = pd.merge(classroom, 
                             courses[['Course_id', 'Course_name', 'Department_id']], 
                             on='Course_id')

#inner join classroom_courses with departments 
classroom_courses_departments = pd.merge(classroom_courses,
                                        departments[['Department_id', 'Department_name']],
                                        on='Department_id')

#counting the number of students in each department
classroom_courses_departments.groupby('Department_id').agg({'Student_id': pd.Series.nunique})

,Student_id
Department_id,
1,84
2,200
3,137
4,81


# 2. How many students have each course of the English department and the total number of students in the department?

In [39]:
# selecting only the English department
english_dep = classroom_courses_departments.copy()
english_dep=english_dep[classroom_courses_departments.Department_id<2]
#counting the number of student in each course
english_dep.groupby(['Department_id','Course_name']).agg({'Student_id': pd.Series.nunique})

Student_id
Department_id Course_name                     
1             Advanced English              24
              English Begginers             32
              Proffesional English          38

In [40]:

#counting total student in the english department
TotalStudent = english_dep.groupby('Department_id').agg({'Student_id': pd.Series.nunique})
TotalStudent

,Student_id
Department_id,
1,84


In [38]:
היי תומאס, מדוע אינני מצליחה לאחד את הטבלאות?
זה מה שהרצתי:
#english_dep.reset_index(inplace=True)
#TotalStudent.reset_index(inplace=True)
#TotalStudent_english = pd.merge(english_dep,TotalStudent, on='Department_id')
#TotalStudent_english.head()

SyntaxError: invalid syntax (<ipython-input-38-9c680c16aedd>, line 1)

# 3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [41]:
#selecting the Science department and agg. the student in each course
science_class =  classroom_courses_departments[classroom_courses_departments.Department_id.isin([2])].\
groupby(['Department_id','Course_id']).\
agg({'Student_id':pd.Series.nunique }) 
science_class.columns=['TotalInClass']


#class size by categ
science_class_cat = science_class.copy()
science_class_cat['ClassSize'] = ''
science_class_cat.loc[(science_class_cat['TotalInClass']< 22),'ClassSize']='Small'
science_class_cat.loc[(science_class_cat['TotalInClass']> 21),'ClassSize']='Big'


#counting the number of each class size
science_class_cat.groupby('ClassSize').agg({'ClassSize':['count']})

,ClassSize
,count
ClassSize,
Big,10
Small,6


# 4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [42]:
#counting the number of each gender 
students.groupby('Gender').agg({'Gender':['count']})

,Gender
,count
Gender,
F,165
M,115


# 5. For which courses the percentage of male/female students is over 70%?

In [51]:
#join classroom_courses with students
courses_classroom_students = pd.merge(classroom_courses,
                            students[['Gender','Student_id']],
                            on='Student_id')

#agg of student gender per course
corse_gender = courses_classroom_students.copy()
corse_gender = corse_gender.groupby(['Course_name','Gender']).\
agg({'Student_id':pd.Series.nunique})
corse_gender.columns=['TotalGender']

#creating a new column containing the total per course
corse_gender['TotalInClass'] = corse_gender.groupby('Course_name').transform('sum')

#creating a new column containing the percentage
corse_gender['Ratio'] = corse_gender.TotalGender / corse_gender.TotalInClass

#selecting the >70% courses
corse_gender = corse_gender[corse_gender.Ratio>0.7]
corse_gender.head(10)

#presenting the %
corse_gender['Percent'] = corse_gender['Ratio']*100
corse_gender.head(10)

,,TotalGender,TotalInClass,Ratio,Percent
Course_name,Gender,,,,
Genetics,F,26,37,0.702703,70.270270
Sculpture,F,17,23,0.739130,73.913043
Tenis,F,13,18,0.722222,72.222222


## Q6. For each department, how many students passed with a grades over 80?

In [47]:
#merge of class with courses
classroom_courses_degree = pd.merge(classroom, 
                             courses[['Course_id', 'Course_name', 'Department_id']], 
                             on='Course_id')

#selecting the student who have grade over 80
degree_80 = classroom_courses_degree[classroom_courses_degree.Degree>80]

#counting the student
degree_80 = degree_80.groupby('Department_id').agg({'Student_id':pd.Series.nunique})
degree_80.head()


,Student_id
Department_id,
1,40
2,122
3,61
4,39


## Q7. For each department, how many students passed with a grades under 60?

In [48]:
#selecting the student who have grade under 60
degree_60 = classroom_courses_degree[classroom_courses_degree.Degree<60]

#counting the student
degree_60 = degree_60.groupby('Department_id').agg({'Student_id':pd.Series.nunique})
degree_60.head()

,Student_id
Department_id,
1,2
2,7
3,3
4,1


# 8. Rate the teachers by their average student's grades (in descending order).

In [49]:
#shorter way of merging 3 tables
teacher_degree = pd.merge(pd.merge(teachers,courses, on='Teacher_id'),classroom, on='Course_id')
        
#calculting the avarage of each teacher
teacher_deg = teacher_degree.groupby(['Teacher_id','First_name','Last_name']).\
agg({'Degree':'mean'})

#sort by degree(descending)
teacher_deg.sort_values(ascending=False,by='Degree')


,,,Degree
Teacher_id,First_name,Last_name,
22,Darick,Hess,81.432703
3,Jordyn,Baron,80.729494
5,Heather,Metcalf,80.440242
20,Zachary,Healy,79.365867
1,Kaylea,Monzingo,79.365537
18,Daniel,Bland,78.908281
24,Keegan,Price,78.814631
15,Conor,Mckittrick,78.699492
23,Maxwell,Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [67]:
#merging the tables
df1_cour_dep = pd.merge(pd.merge(pd.merge(courses,
                       departments,
                       on='Department_id'),teachers,on='Teacher_id'),classroom,on='Course_id')

df1_cour_dep['Teachers_name'] = df1_cour_dep.First_name +' '+ df1_cour_dep.Last_name
#grouping by courses and counting the number of students in each course
df2_cour_dep = df1_cour_dep.copy()
df2_cour_dep = df2_cour_dep.groupby(['Course_name','Department_name','Teachers_name']).agg({'Student_id':'count'}) 

df2_cour_dep.head()


,,,Student_id
Course_name,Department_name,Teachers_name,
Advanced English,English,Conor Mckittrick,24
Algebra,Science,Alexander Watts,12
Biology,Science,Jordyn Baron,36
Chemistry,Science,Jeffrey Batzel,24
Computer Science,Science,Kaylea Monzingo,38


# 10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [76]:
students['Student_name'] = students.First_name +' '+ students.Last_name
df3_student_course = pd.merge(pd.merge(students[['Student_name','Student_id']],classroom[['Course_id']],on='Student_id')courses,on='Course_id')
df3_student_course.head()


SyntaxError: invalid syntax (<ipython-input-76-db9e23ff385c>, line 2)